## FINAL PHASE: GOLD LAYER & DASHBOARDS.

At this stage, no more raw FHIR complexity.
Gold = business-ready, aggregated, interview-friendly tables.

I’ll structure this clearly:

1️⃣ What Gold layer means (very short)

2️⃣ Gold tables for each dashboard (logic)

3️⃣ Exact SQL code for each Gold table

4️⃣ How to build dashboards in Databricks SQL (Free 
Tier)

5️⃣ How to explain this in interviews

🧠 WHAT YOU HAVE NOW (VERY IMPORTANT)

You now have ALL critical Silver tables:

✔ Patient
✔ Encounter
✔ Condition
✔ Observation
✔ Procedure
✔ MedicationRequest
✔ Immunization
✔ DiagnosticReport
✔ ExplanationOfBenefit

###🧱 GOLD VIEW WITH ALL RESOURCE TYPES (9–10 TABLES)

You now have these Silver tables:

patient

encounter

condition

observation

explanation_of_benefit

procedure

medication_request

immunization

diagnostic_report

Below is a correct, production-grade Gold view joining ALL of them.

### GOLD VIEW (CORRECT & COMPLETE)

In [0]:
%sql
SELECT *
FROM healthcare.fhir_healthcare_analytics_gold.patient_encounter_summary
LIMIT 20;


### 🥇 GOLD DASHBOARD 1: PATIENT ADMISSIONS
Business Question

How many admissions?

Daily / monthly trends?

Average length of stay?

### 🟨 GOLD TABLE: gold_patient_admissions

📊 Dashboard visuals

Line chart → Admissions over time

KPI → Avg length of stay

SQL (Databricks SQL)


In [0]:
%sql
CREATE OR REPLACE TABLE healthcare.fhir_healthcare_analytics_gold.patient_admissions AS
SELECT
  DATE(e.admit_time) AS admission_date,
  COUNT(DISTINCT e.encounter_id) AS total_admissions,
  AVG(
    DATEDIFF(
      DATE(e.discharge_time),
      DATE(e.admit_time)
    )
  ) AS avg_length_of_stay
FROM healthcare.fhir_healthcare_analytics_silver.encounter e
WHERE e.admit_time IS NOT NULL
GROUP BY DATE(e.admit_time)
ORDER BY admission_date;

### 🥇 GOLD DASHBOARD 2:  DISEASE PATTERNS

Business Question

Most common diseases?

Trends over time?

#### 🟨 GOLD TABLE: gold_disease_patterns

📊 Dashboard visuals

Bar chart → Top diagnoses

Table → Diagnosis vs patient count


In [0]:
%sql
CREATE OR REPLACE TABLE healthcare.fhir_healthcare_analytics_gold.disease_patterns AS
SELECT
  c.diagnosis,
  COUNT(DISTINCT c.patient_id) AS patient_count,
  COUNT(DISTINCT c.encounter_id) AS encounter_count
FROM healthcare.fhir_healthcare_analytics_silver.condition c
WHERE c.diagnosis IS NOT NULL
GROUP BY c.diagnosis
ORDER BY patient_count DESC;

### 🥇 GOLD DASHBOARD 3: TREATMENT TRENDS
Business Question

What treatments are performed most?

What medications are prescribed most?

🟨 GOLD TABLE: PROCEDURE TRENDS

📊 Dashboard visuals

Bar chart → Top procedures

Bar chart → Top medications

In [0]:
%sql
CREATE OR REPLACE TABLE healthcare.fhir_healthcare_analytics_gold.procedure_trends AS
SELECT
  p.procedure_name,
  COUNT(*) AS procedure_count
FROM healthcare.fhir_healthcare_analytics_silver.procedure p
WHERE p.procedure_name IS NOT NULL
GROUP BY p.procedure_name
ORDER BY procedure_count DESC;



🟨 GOLD TABLE: MEDICATION TRENDS


In [0]:
%sql
CREATE OR REPLACE TABLE healthcare.fhir_healthcare_analytics_gold.medication_trends AS
SELECT
  m.medication_name,
  COUNT(*) AS prescription_count
FROM healthcare.fhir_healthcare_analytics_silver.medication_request m
WHERE m.medication_name IS NOT NULL
GROUP BY m.medication_name
ORDER BY prescription_count DESC;

### 🥇 GOLD DASHBOARD 4: COST ANALYSIS
Business Question

Cost per encounter?

Cost drivers?

🟨 GOLD TABLE: gold_cost_analysis

📊 Dashboard visuals

KPI → Avg cost per encounter

Bar chart → High-cost encounters

In [0]:
%sql
CREATE OR REPLACE TABLE healthcare.fhir_healthcare_analytics_gold.cost_analysis AS
SELECT
  e.encounter_id,
  e.patient_id,
  SUM(
    CAST(eob.total_amount AS DOUBLE)
  ) AS total_cost
FROM healthcare.fhir_healthcare_analytics_silver.explanation_of_benefit eob
JOIN healthcare.fhir_healthcare_analytics_silver.encounter e
  ON eob.encounter_id = e.encounter_id
GROUP BY e.encounter_id, e.patient_id
ORDER BY total_cost DESC;

### 🥇 GOLD DASHBOARD 5: PREVENTIVE CARE (IMMUNIZATION)
Business Question

How many vaccinations?

Vaccine usage patterns?

🟨 GOLD TABLE: gold_immunization_coverage

📊 Dashboard visuals

Bar chart → Vaccine coverage

KPI → Total vaccinated patients

In [0]:
%sql
CREATE OR REPLACE TABLE healthcare.fhir_healthcare_analytics_gold.immunization_coverage AS
SELECT
  i.vaccine_name,
  COUNT(DISTINCT i.patient_id) AS vaccinated_patients
FROM healthcare.fhir_healthcare_analytics_silver.immunization i
WHERE i.vaccine_name IS NOT NULL
GROUP BY i.vaccine_name
ORDER BY vaccinated_patients DESC;

In [0]:
%sql
-- Encounter count in last 30 days
SELECT
  patient_id,
  COUNT(*) AS encounter_count_30d
FROM healthcare.fhir_healthcare_analytics_gold.patient_encounter_summary
WHERE prescribed_date >= current_date() - INTERVAL 30 DAYS
GROUP BY patient_id;

